In [1]:
import numpy as np
import pandas
import turicreate

ImportError: ignored

In [2]:
class popularity_recommender():
    def __init__(self):
        self.t_data = None                                
        self.u_id = None                    #ID of the user
        self.i_id = None                    #ID of Song the user is listening to
        self.pop_recommendations = None     #getting popularity recommendations according to that
        
    #Create the system model
    def create_p(self, t_data, u_id, i_id):
        self.t_data = t_data
        self.u_id = u_id
        self.i_id = i_id
        
        #Get the no. of times each song has been listened as recommendation score 
        t_data_grouped = t_data.groupby([self.i_id]).agg({self.u_id: 'count'}).reset_index()
        t_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
    
        #Sort the songs based upon recommendation score
        t_data_sort = t_data_grouped.sort_values(['score', self.i_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        t_data_sort['Rank'] = t_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.pop_recommendations = t_data_sort.head(10)
    
    #Use the system model to give recommendations
    def recommend_p(self, u_id):    
        u_recommendations = self.pop_recommendations
        
        #Add user_id column for which the recommended songs are generated
        u_recommendations['user_id'] = u_id
    
        #Bring user_id column to the front
        cols = u_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        u_recommendations = u_recommendations[cols]
        
        return u_recommendations

In [3]:
class similarity_recommender():
    def __init__(self):
        self.t_data = None
        self.u_id = None
        self.i_id = None
        self.co_matrix = None
        self.songs_dic = None
        self.rev_songs_dic = None
        self.i_similarity_recommendations = None
        
    #Get unique songs corresponding to a given user
    def get_user_items(self, u):
        u_data = self.t_data[self.t_data[self.u_id] == u]
        u_items = list(u_data[self.i_id].unique())
        
        return u_items
        
    #Get unique users for a given song
    def get_i_users(self, i):
        i_data = self.t_data[self.t_data[self.i_id] == i]
        i_users = set(i_data[self.u_id].unique())
            
        return i_users
        
    #Get unique songs in the training data
    def get_all_items_t_data(self):
        all_items = list(self.t_data[self.i_id].unique())
            
        return all_items
        
    #Construct cooccurence matrix
    def construct_co_matrix(self, u_songs, a_songs):
            
        #Get users for all songs in user_songs.
        u_songs_users = []        
        for i in range(0, len(u_songs)):
            u_songs_users.append(self.get_i_users(u_songs[i]))
            
        #Initialize the item cooccurence matrix of size len(user_songs) X len(songs)
        co_matrix = np.matrix(np.zeros(shape=(len(u_songs), len(a_songs))), float)
           
        #Calculate similarity between songs listened by the user and all unique songs in the training data
        for i in range(0,len(a_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.t_data[self.t_data[self.i_id] == a_songs[i]]
            users_i = set(songs_i_data[self.u_id].unique())
            
            for j in range(0,len(u_songs)):       
                    
                #Get unique listeners (users) of song (item) j
                users_j = u_songs_users[j]
                    
                #Calculate the songs which are in common listened by users i & j
                users_intersection = users_i.intersection(users_j)
                
                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate all the songs listened by i & j
                    users_union = users_i.union(users_j)
                    
                    co_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    co_matrix[j,i] = 0
                    
        
        return co_matrix
    
    #Use the cooccurence matrix to make top recommendations
    def generate_top_r(self, user, cooccurence_matrix, a_songs, u_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))
        
        #Calculate the average of the scores in the cooccurence matrix for all songs listened by the user.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()
 
        #Sort the indices of user_sim_scores based upon their value also maintain the corresponding score
        s_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)
    
        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df1 = pandas.DataFrame(columns=columns)
         
        #Fill the dataframe with top 10 songs
        rank = 1 
        for i in range(0,len(s_index)):
            if ~np.isnan(s_index[i][0]) and a_songs[s_index[i][1]] not in u_songs and rank <= 10:
                df1.loc[len(df1)]=[user,a_songs[s_index[i][1]],s_index[i][0],rank]
                rank = rank+1
        
        #Handle the case where there are no recommendations
        if df1.shape[0] == 0:
            print("The current user don't have any song for similarity based recommendation model.")
            return -1
        else:
            return df1
 
    #Create the system model
    def create_s(self, t_data, u_id, i_id):
        self.t_data = t_data
        self.u_id = u_id
        self.i_id = i_id
    #Use the model to make recommendations
    def recommend_s(self, u):
        
        #A. Get all unique songs for this user
        u_songs = self.get_user_items(u)    
            
        print("No. of songs for the user: %d" % len(u_songs))
    
        #B. Get all the songs in the data
        a_songs = self.get_all_items_t_data()
        
        print("No. of songs in the list: %d" % len(a_songs))
         
        #C. Make the cooccurence matrix of size len(user_songs) X len(songs)
        co_matrix = self.construct_co_matrix(u_songs, a_songs)
        
        #D. Use the matrix to make recommended songs
        df_r = self.generate_top_r(u, co_matrix, a_songs, u_songs)
        return df_r
    
    #Create a function to get similar songs
    def similar_items(self, i_list):
        
        u_songs = i_list
        
        #A. Get all the songs from the data
        a_songs = self.get_all_items_t_data()
        
        print("no. of unique songs in the set: %d" % len(a_songs))
         
        #B. Make the cooccurence matrix of size len(user_songs) X len(songs)
        co_matrix = self.construct_co_matrix(u_songs, a_songs)
        
        #C. Use the matrix to make recommendations
        u = ""
        df_r = self.generate_top_r(u, co_matrix, a_songs, u_songs)
         
        return df_r

In [4]:
import numpy as np
import pandas
import turicreate
from sklearn.model_selection import train_test_split
import time
from popularity_recommender import popularity_recommender
from similarity_recommender import similarity_recommender

ImportError: ignored

In [ ]:
triplets = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata = 'https://static.turi.com/datasets/millionsong/song_data.csv'

song_df_a = pandas.read_table(triplets,header=None)
song_df_a.columns = ['user_id', 'song_id', 'listen_count']

#Read song  metadata
song_df_b =  pandas.read_csv(songs_metadata)

#Merge the two dataframes above to create input dataframe for recommender systems
song_df1 = pandas.merge(song_df_a, song_df_b.drop_duplicates(['song_id']), on="song_id", how="left")
song_df1.head()



/Users/prashanttrupti/opt/anaconda3/envs/venv-2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [ ]:
print("Total no of songs:",len(song_df1))

('Total no of songs:', 2000000)


In [ ]:

song_df1 = song_df1.head(20000)
stats = turicreate.SFrame(song_df1)
turicreate.show(stats[1:5000]['title'],stats[1:5000]['count'])
#Merge song title and artist_name columns to make a new column
song_df1['song'] = song_df1['title'].map(str) + " - " + song_df1['artist_name']



RuntimeError: Column name count does not exist.

In [ ]:
song_gr = song_df1.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_gr['listen_count'].sum()
song_gr['percentage']  = song_gr['listen_count'].div(grouped_sum)*100
song_gr.sort_values(['listen_count', 'song'], ascending = [0,1])

,song,listen_count,percentage
5183,Sehr kosmisch - Harmonia,79,0.395
1514,Dog Days Are Over (Radio Edit) - Florence + Th...,69,0.345
4918,Revelry - Kings Of Leon,65,0.325
6603,Undo - Björk,65,0.325
5175,Secrets - OneRepublic,64,0.320
...,...,...,...
7226,Zwitter - Rammstein,1,0.005
7228,aNYway - Armand Van Helden & A-TRAK Present Du...,1,0.005
7230,high fives - Four Tet,1,0.005
7231,in white rooms - Booka Shade,1,0.005


In [ ]:
u = song_df1['user_id'].unique()
print("The no. of unique users:", len(u))

The no. of unique users: 756


In [ ]:
train, test_data = train_test_split(song_df1, test_size = 0.20, random_state=0)
print(train.head(5))

                                        user_id             song_id  \
17815  270dffee86b3e7bfb81892f5ae2c60c1d2f15839  SOEOJHS12AB017F3DC   
18370  ed0f4979ffddc0010244638a3b956510624dc6a9  SOETQSU12A58A7E2A0   
1379   3f152d355d53865a2ca27ac5ceeffb7ebaea0a26  SOKLVUI12A6701BF1B   
14763  cc62e608a1a21353e1673817973235cde4116bad  SOLHXPC12A8C142CC6   
7346   94d5bdc37683950e90c56c9b32721edb5d347600  SONQBUB12A6D4F8ED0   

       listen_count                          title  \
17815             1                 Help I'm Alive   
18370             1                       Mix Tape   
1379              8                     If I Could   
14763             1                     Back To Me   
7346              3  Angie (1993 Digital Remaster)   

                                                 release  \
17815                                          Fantasies   
18370                               Your Favorite Weapon   
1379                                   In Between Dreams   
14763   

In [ ]:
pm = popularity_recommender()                               #create an instance of the class
pm.create_p(train, 'user_id', 'song')
user_id1 = u[5]                                                          #Recommended songs list for a user
pm.recommend_p(user_id1)

,user_id,song,score,Rank
4692,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Sehr kosmisch - Harmonia,66,1.0
1338,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Dog Days Are Over (Radio Edit) - Florence + Th...,56,2.0
4445,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Revelry - Kings Of Leon,56,3.0
6520,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You're The One - Dwight Yoakam,51,4.0
4684,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Secrets - OneRepublic,47,5.0
5988,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Undo - Björk,47,6.0
1753,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Fireflies - Charttraxx Karaoke,44,7.0
2355,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Horn Concerto No. 4 in E flat K495: II. Romanc...,40,8.0
5615,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Scientist - Coldplay,40,9.0
5824,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Tive Sim - Cartola,37,10.0


In [ ]:
user_id2 = u[7]
pm.recommend_p(user_id2)

,user_id,song,score,Rank
4692,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Sehr kosmisch - Harmonia,66,1.0
1338,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Dog Days Are Over (Radio Edit) - Florence + Th...,56,2.0
4445,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Revelry - Kings Of Leon,56,3.0
6520,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,You're The One - Dwight Yoakam,51,4.0
4684,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Secrets - OneRepublic,47,5.0
5988,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Undo - Björk,47,6.0
1753,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Fireflies - Charttraxx Karaoke,44,7.0
2355,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Horn Concerto No. 4 in E flat K495: II. Romanc...,40,8.0
5615,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,The Scientist - Coldplay,40,9.0
5824,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Tive Sim - Cartola,37,10.0


In [ ]:
is_model = similarity_recommender()
is_model.create_s(train, 'user_id', 'song')

In [ ]:
user_id2 = u[7]
#Fill in the code here
user_items2 = is_model.get_user_items(user_id2)
print("------------------------------------------------------------------------------------")
print("Songs played by second user %s:" % user_id2)
print("------------------------------------------------------------------------------------")

for user_item in user_items2:
    print(user_item)

print("----------------------------------------------------------------------")
print("Similar songs recommended for the second user:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend_s(user_id2)

------------------------------------------------------------------------------------
Songs played by second user 9d6f0ead607ac2a6c2460e4d14fb439a146b7dec:
------------------------------------------------------------------------------------
Speed Of Sound - Coldplay
Yellow - Coldplay
The Scientist - Coldplay
Clocks - Coldplay
Swallowed In The Sea - Coldplay
Life In Technicolor ii - Coldplay
Life In Technicolor - Coldplay
Lost! - Coldplay
Trouble - Coldplay
Strawberry Swing - Coldplay
----------------------------------------------------------------------
Similar songs recommended for the second user:
----------------------------------------------------------------------
No. of songs for the user: 10
No. of songs in the list: 6574
Non zero values in cooccurence_matrix :6785


,user_id,song,score,rank
0,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Now My Feet Won't Touch The Ground - Coldplay,0.188940,1
1,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,The Goldrush - Coldplay,0.188940,2
2,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,All The Kings Horns - Sufjan Stevens,0.188940,3
3,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Warning Sign - Coldplay,0.188940,4
4,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Puppetmad - Puppetmastaz,0.184524,5
5,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,The Joker - Fatboy Slim,0.184524,6
6,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Korg Rhythm Afro - Holy Fuck,0.184524,7
7,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,high fives - Four Tet,0.184524,8
8,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,You Only Live Twice (Live From Norway) - Coldplay,0.184524,9
9,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Take It In - Hot Chip,0.184524,10


In [ ]:
is_model.similar_items(['U Smile - Justin Bieber'])

no. of unique songs in the set: 6574
Non zero values in cooccurence_matrix :565


,user_id,song,score,rank
0,,That Should Be Me - Justin Bieber,0.312500,1
1,,Somebody To Love - Justin Bieber,0.307692,2
2,,You've Got The Love - Florence + The Machine,0.272727,3
3,,What Goes Around...Comes Around - Justin Timbe...,0.272727,4
4,,One Time - Justin Bieber,0.263158,5
5,,Mia - Emmy The Great,0.263158,6
6,,Paper Planes - M.I.A.,0.260870,7
7,,Eenie Meenie - Sean Kingston and Justin Bieber,0.260870,8
8,,Lucky (Album Version) - Jason Mraz & Colbie Ca...,0.258065,9
9,,Heartbreak Warfare - John Mayer,0.240000,10
